If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers sacrebleu

     |████████████████████████████████| 90 kB 1.3 MB/s eta 0:00:01


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
# !apt install git-lfs

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


4.12.3


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](https://github.com/huggingface/notebooks/blob/master/examples/images/translation.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [5]:
model_checkpoint = "Helsinki-NLP/opus-mt-de-is"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [7]:
from datasets import load_dataset, load_metric

fra_doods

dataset_de = load_dataset("gsarti/flores_101", "deu")
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/278k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset flores101 downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/gsarti___flores101/deu/1.0.0/22346864a94723ccfc3ac88520a7348d91f8d08fa4d1929a7d719b524802c467. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [8]:
dataset_de

DatasetDict({
    dev: Dataset({
        features: ['id', 'sentence', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink'],
        num_rows: 997
    })
    devtest: Dataset({
        features: ['id', 'sentence', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink'],
        num_rows: 1012
    })
})

In [11]:
dataset_is = load_dataset("gsarti/flores_101", "isl")

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset flores101 downloaded and prepared to /home/oddurace/.cache/huggingface/datasets/gsarti___flores101/isl/1.0.0/22346864a94723ccfc3ac88520a7348d91f8d08fa4d1929a7d719b524802c467. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

To access an actual element, you need to select a split first, then give an index:

In [17]:
from pprint import pprint
pprint(dataset_de['dev'][1])
pprint(dataset_is['dev'][1])

{'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'has_hyperlink': 0,
 'has_image': 0,
 'id': 2,
 'sentence': 'Führende Forscher sagen, dass dies die Früherkennung von Krebs, '
             'Tuberkulose, HIV und Malaria für Patienten in '
             'einkommensschwachen Ländern fördern könnte, wo die '
             'Überlebensraten bei Krankheiten wie Brustkrebs teilweise nur '
             'halb so hoch sind wie in reicheren Ländern.',
 'topic': 'health'}
{'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'has_hyperlink': 0,
 'has_image': 0,
 'id': 2,
 'sentence': 'Leiðandi vísindamenn segja að þetta geti orðið til snemmbúinnar '
             'greiningar á krabbameini, berklum, alnæmi og malaríu hjá '
             'sjúklingum í lágtekjulöndum, þar sem lifunarhlutfall fyrir '
             'sjúkdó

In [61]:
dataset_de1 = dataset_de.remove_columns(['URL', 'domain', 'has_hyperlink', 'has_image', 'topic'])
dataset_is1 = dataset_is.remove_columns(['URL', 'domain', 'has_hyperlink', 'has_image', 'topic'])

In [62]:
dataset_de1 = dataset_de1.rename_column('sentence', 'de')
dataset_is1 = dataset_is1.rename_column('sentence', 'is')

In [63]:
print(dataset_de1)
print(dataset_is1)

DatasetDict({
    dev: Dataset({
        features: ['id', 'de'],
        num_rows: 997
    })
    devtest: Dataset({
        features: ['id', 'de'],
        num_rows: 1012
    })
})
DatasetDict({
    dev: Dataset({
        features: ['id', 'is'],
        num_rows: 997
    })
    devtest: Dataset({
        features: ['id', 'is'],
        num_rows: 1012
    })
})


In [64]:
#hakk
pd_de = dataset_de1['dev'].to_pandas()
pd_is = dataset_is1['dev'].to_pandas()

In [65]:
print(pd_is)

      id                                                 is
0      1  Á mánudag tilkynntu vísindamenn frá læknadeild...
1      2  Leiðandi vísindamenn segja að þetta geti orðið...
2      3  JAS 39C Gripen-flugvélin brotlenti á flugbraut...
3      4  Staðfest var að flugmaðurinn var leiðtogi flok...
4      5  Samkvæmt fjölmiðlum á staðnum valt slökkvibíl ...
..   ...                                                ...
992  993  Á indverska sumrinu nær ferðamannatími hæðarst...
993  994  Slík fjallþorp búa hins vegar yfir annarri feg...
994  995  Aðeins fá flugfélög bjóða enn gjöld vegna ástv...
995  996  Flugfélög sem bjóða upp á slíkt eru m.a. Air C...
996  997  Þú verður alltaf að bóka beint hjá flugfélagin...

[997 rows x 2 columns]


In [66]:
import pandas as pd
megahakk = pd.merge(pd_de, pd_is, on='id')

In [68]:
print(megahakk)

      id                                                 de  \
0      1  Am Montag haben die Wisenschaftler der Stanfor...   
1      2  Führende Forscher sagen, dass dies die Früherk...   
2      3  Der JAS 39C Gripen stürzte gegen 9:30 Uhr Orts...   
3      4  Der Pilot wurde als Staffelführer Dilokrit Pat...   
4      5  Lokale Medien berichten, dass ein Feuerwehrwag...   
..   ...                                                ...   
992  993  Die Touristensaison für die Hügelstationen err...   
993  994  Allerdings verfügen sie im Winter über eine an...   
994  995  Nur noch wenige Fluggesellschaften bieten Tode...   
995  996  Zu den Fluggesellschaften, die diese anbieten,...   
996  997  Sie müssen in jedem Fall telefonisch über die ...   

                                                    is  
0    Á mánudag tilkynntu vísindamenn frá læknadeild...  
1    Leiðandi vísindamenn segja að þetta geti orðið...  
2    JAS 39C Gripen-flugvélin brotlenti á flugbraut...  
3    Staðfest v

In [102]:
from datasets import Dataset
hello = Dataset.from_pandas(megahakk)

In [103]:
print(hello)

Dataset({
    features: ['id', 'de', 'is', '__index_level_0__'],
    num_rows: 997
})


In [104]:
wow = hello.remove_columns(['__index_level_0__'])

In [105]:
for each in range(wow.num_rows):
    print(wow[each], "\n")

{'id': 1, 'de': 'Am Montag haben die Wisenschaftler der Stanford University School of Medicine die Erfindung eines neuen Diagnosetools bekanntgegeben, mit dem Zellen nach ihrem Typ sortiert werden können: ein winziger, ausdruckbarer Chip, der für jeweils etwa einen US-Cent mit Standard-Tintenstrahldruckern hergestellt werden kann.', 'is': 'Á mánudag tilkynntu vísindamenn frá læknadeild Stanford-háskóla uppfinningu á nýju greiningartæki sem getur flokkað frumur eftir tegund: örlítill prentanleg flaga sem hægt er að framleiða með venjulegum bleksprautuprentara fyrir mögulega um eitt bandarískt sent stykkið.'} 

{'id': 2, 'de': 'Führende Forscher sagen, dass dies die Früherkennung von Krebs, Tuberkulose, HIV und Malaria für Patienten in einkommensschwachen Ländern fördern könnte, wo die Überlebensraten bei Krankheiten wie Brustkrebs teilweise nur halb so hoch sind wie in reicheren Ländern.', 'is': 'Leiðandi vísindamenn segja að þetta geti orðið til snemmbúinnar greiningar á krabbameini, b

In [110]:
print(wow)

Dataset({
    features: ['id', 'de', 'is'],
    num_rows: 997
})


In [112]:
print(wow[1]['is'])

Leiðandi vísindamenn segja að þetta geti orðið til snemmbúinnar greiningar á krabbameini, berklum, alnæmi og malaríu hjá sjúklingum í lágtekjulöndum, þar sem lifunarhlutfall fyrir sjúkdóma á borð við brjóstakrabbamein getur verið helmingi lægri en hjá ríkari löndum.


In [123]:
from datasets import DatasetDict

#gera translations til að passa við formið
def get_transDict(dataset):
    transDict = { 'id': [], 'translation': [] }
    for item in dataset:
        transDict['id'].append(item['id'])
        transDict['translation'].append({'de': item['de'], 'is': item['is']})
    transDataset = Dataset.from_dict(transDict)
    transData = DatasetDict({'train': transDataset})
    return transData

In [124]:
virkaretta = get_transDict(wow)

In [125]:
kommon = DatasetDict(virkaretta)

In [126]:
print(kommon)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 997
    })
})


In [130]:
print(kommon['train'][1]['translation'])

{'de': 'Führende Forscher sagen, dass dies die Früherkennung von Krebs, Tuberkulose, HIV und Malaria für Patienten in einkommensschwachen Ländern fördern könnte, wo die Überlebensraten bei Krankheiten wie Brustkrebs teilweise nur halb so hoch sind wie in reicheren Ländern.', 'is': 'Leiðandi vísindamenn segja að þetta geti orðið til snemmbúinnar greiningar á krabbameini, berklum, alnæmi og malaríu hjá sjúklingum í lágtekjulöndum, þar sem lifunarhlutfall fyrir sjúkdóma á borð við brjóstakrabbamein getur verið helmingi lægri en hjá ríkari löndum.'}


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [88]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [89]:
show_random_elements(raw_datasets["train"])

NameError: name 'raw_datasets' is not defined

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments)
    references: A list of one or more reference streams (each a sequence of segments)
    smooth: The smoothing method to use
    smooth_value: For 'floor' smoothing, the floor to use
    force: Ignore data that looks already tokenized
    lowercase: Lowercase the data
    tokenize: The tokenizer to use
Returns:
    'score': BLEU score,
    'counts': Counts,
    'totals': Totals,
    'precisions': Precisions,
    'bp': Brevity penalty,
    'sys_len': predictions length,
    'ref_len': reference length,
Examples:

    >>> predictions = ["hello there general kenobi", "foo bar foobar"]
    >>> references = [["hello there gen

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [125, 778, 3, 63, 141, 9191, 23, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[125, 778, 3, 63, 141, 9191, 23, 0], [187, 32, 716, 9191, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], [235, 1705, 11, 32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[393, 4462, 14, 1137, 53, 216, 28636, 0], [24385, 14, 28636, 14, 4646, 4622, 53, 216, 28636, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/9dc00622c30446e99c4c63d12a484ea4fb653f2f37c867d6edcec839d7eae50f/cache-126b7925258542cf.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/9dc00622c30446e99c4c63d12a484ea4fb653f2f37c867d6edcec839d7eae50f/cache-995e36d1e1745505.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/9dc00622c30446e99c4c63d12a484ea4fb653f2f37c867d6edcec839d7eae50f/cache-313c89c543a3c175.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target-lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/marian-finetuned-en-to-ro"` or `"huggingface/marian-finetuned-en-to-ro"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len,Runtime,Samples Per Second
1,0.740100,1.290665,28.059300,34.051500,135.611700,14.741000


TrainOutput(global_step=38145, training_loss=0.7717826230230017, metrics={'train_runtime': 4738.3882, 'train_samples_per_second': 8.05, 'total_flos': 3.62019881263104e+16, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 337319, 'init_mem_gpu_alloc_delta': 300833792, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1028051, 'train_mem_gpu_alloc_delta': 899235328, 'train_mem_cpu_peaked_delta': 267371462, 'train_mem_gpu_peaked_delta': 3136797184})

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```